In [ ]:
from adler.objectdata.AdlerPlanetoid import AdlerPlanetoid
from adler.science.PhaseCurve import PhaseCurve
from adler.objectdata.AdlerData import AdlerData
from adler.utilities.plotting_utilities import plot_errorbar
import adler.utilities.science_utilities as sci_utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import astropy.units as u

In [ ]:
# ssoid = "6098332225018"  # good test object
ssoid = "8268570668335894776"  # NEO

fname = "/Users/jrobinson/lsst-adler/notebooks/gen_test_data/adler_demo_testing_database.db"
planetoid = AdlerPlanetoid.construct_from_SQL(ssoid, sql_filename=fname)

In [ ]:
planetoid.__dict__

In [ ]:
planetoid.MPCORB

In [ ]:
filters = planetoid.filter_list

In [ ]:
# get all obs as dataframe
df_obs = pd.DataFrame()
for filt in filters:
    obs = planetoid.observations_in_filter(filt)
    _df_obs = pd.DataFrame(obs.__dict__)
    df_obs = pd.concat([df_obs, _df_obs])
    df_obs = df_obs.reset_index(drop=True)
#     break

In [ ]:
df_obs

In [ ]:
tmin = np.amin(np.floor(df_obs["midPointMjdTai"]))  # mjd
tmax = np.amax(np.floor(df_obs["midPointMjdTai"])) + 1  # mjd
tmin, tmax

In [ ]:
# cumulative data in filter
x_plot = "midPointMjdTai"
df_plot = df_obs.sort_values(x_plot)

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

bins = np.arange(tmin, tmax + 1)

values, base = np.histogram(df_plot[x_plot], bins=bins)
cumulative = np.cumsum(values)
ax1.plot(base[:-1] - base[0], cumulative)

data_mask = np.diff(cumulative) > 0
data_nights = base[1:-1][data_mask]
N_data = cumulative[1:][data_mask]

ax1.scatter(data_nights - data_nights[0], N_data)

ax1.set_xlabel(x_plot)
ax1.set_ylabel("number")

plt.show()

In [ ]:
data_nights

In [ ]:
N_data

In [ ]:
# create the empty AdlerData object
adler_data = AdlerData(ssoid, planetoid.filter_list)
adler_data

In [ ]:
planetoid.SSObject

In [ ]:
planetoid.SSObject_in_filter("r")

In [ ]:
# TODO: we need a translation between planetoid.SSObject attributes and AdlerData attributes
ADLER_SSOBJECT = {"phase_parameter_1": "G12"}
SSOBJECT_ADLER = {v: k for k, v in ADLER_SSOBJECT.items()}

In [ ]:
ADLER_SSOBJECT, SSOBJECT_ADLER

In [ ]:
sso_r = planetoid.SSObject_in_filter("r")
sso_r.H, sso_r.G12

In [ ]:
N_pc_fit = 25  # minimum number of observations to fit phase curve parameter (otherwise just fit H)
mod_name = "HG12_Pen16"  # model name

# create the empty AdlerData object
adler_data = AdlerData(ssoid, planetoid.filter_list)

ad_params_list = []

for n in data_nights:
    print(n)
    df_n = df_obs[df_obs["midPointMjdTai"] < n]
    print(len(df_n), np.unique(df_n["filter_name"]))
    print(df_n.value_counts("filter_name"))
    #     break

    for filt in np.unique(df_n["filter_name"]):
        _df_n = df_n[df_n["filter_name"] == filt]
        print(filt, len(_df_n))

        # Try get parameters from AdlerData otherwise, use the SSObject parameters from alert
        try:
            params = adler_data.get_phase_parameters_in_filter(filt, model_name=mod_name)
            print("use AdlerData parameters")
            phase_parameter_1 = "phase_parameter_1"

        except ValueError:
            params = planetoid.SSObject_in_filter(filt)
            print("use SSObject parameters")
            phase_parameter_1 = ADLER_SSOBJECT["phase_parameter_1"]

        print(params)

        if not hasattr(params.H, "unit"):
            params.H *= u.mag

        # Define the initial simple phase curve filter model with fixed G12
        pc = PhaseCurve(
            H=params.H,
            phase_parameter_1=getattr(params, phase_parameter_1),
            model_name=mod_name,
        )

        # do we fix phase parameter?
        if len(_df_n) < N_pc_fit:
            pc.model_function.G12.fixed = True
        else:
            pc.model_function.G12.fixed = False

        # do a HG12_Pen16 fit to the past data
        pc_fit = pc.FitModel(
            np.array(_df_n["phaseAngle"]) * u.deg,
            np.array(_df_n["reduced_mag"]) * u.mag,
            np.array(_df_n["magErr"]) * u.mag,
        )
        pc_fit = pc.InitModelSbpy(pc_fit)
        print(filt, pc_fit.__dict__)

        # define a dict of all values, including metadata
        # TODO: include units?
        # TODO: add a model fit timestamp parameter?
        ad_params = pc_fit.__dict__
        ad_params["phaseAngle_min"] = np.amin(_df_n["phaseAngle"])  # * u.deg
        ad_params["phaseAngle_range"] = np.ptp(_df_n["phaseAngle"])  # * u.deg
        ad_params["arc"] = np.ptp(_df_n["midPointMjdTai"])  # * u.d
        ad_params["nobs"] = len(_df_n)
        ad_params["modelFitMjd"] = np.amax(_df_n["midPointMjdTai"])

        # clean the dict of units
        for x in ad_params:
            if hasattr(ad_params[x], "unit"):
                ad_params[x] = ad_params[x].value

        # store values in AdlerData
        adler_data.populate_phase_parameters(filt, **ad_params)

        ad_params_list.append(ad_params)

#         break

#     break

In [ ]:
fig = plot_errorbar(planetoid, filt_list=["r"])

In [ ]:
ad_params

In [ ]:
# x_plot = "midPointMjdTai"
x_plot = "modelFitMjd"
y_plot = "H"

for y_plot in [
    "H",
    "phase_parameter_1",
    #                "phaseAngle_min",
    #               "phaseAngle_range",
    #               "arc",
    #               "nobs",
]:
    df_plot = pd.DataFrame(ad_params_list)

    fig = plt.figure()
    gs = gridspec.GridSpec(1, 1)
    ax1 = plt.subplot(gs[0, 0])

    #     ax1.scatter(df_plot[x_plot],df_plot[y_plot])
    ax1.scatter(df_plot.index.values, df_plot[y_plot])

    # ax1.set_xlabel(x_plot)
    ax1.set_ylabel(y_plot)

    plt.show()

In [ ]:
_df_n

In [ ]:
df_obs.columns

In [ ]:
# calculate elongation angle
R = df_obs["heliocentricDist"]
Delta = df_obs["topocentricDist"]
alpha = np.radians(df_obs["phaseAngle"])

R_E = np.sqrt((R * R) + (Delta * Delta) - (2.0 * R * Delta * np.cos(alpha)))
df_obs["elong"] = np.degrees(np.arccos(((R_E * R_E) + (Delta * Delta) - (R * R)) / (2.0 * R_E * Delta)))

In [ ]:
x_plot = "midPointMjdTai"
y_plot = "elong"
df_plot = df_obs

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.scatter(df_plot[x_plot], df_plot[y_plot])

ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)

plt.show()